<a href="https://colab.research.google.com/github/KiitanTheAnalyst/Ersilia---Olaitan-Suru/blob/main/Model%20Bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Installing necessary Libraries and specifying path to relevant folders
!pip install rdkit
!pip install standardiser
# In this codeblock I will import the necessary packages and specify the paths to relevant folders
from rdkit import Chem
from rdkit.Chem import AllChem
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys

sys.path.append('../src')
DATAPATH = "../data"
FIGUREPATH = "../figures/"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.4/698.4 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for standardiser: filename=standardiser-0.1.9-py3-none-any.whl size=720100 sha256=81bc0e3c41602da95924f5d8e7b52f1a9ea49442b27aef65a91e0cd5a0ebf969
  Stored in directory: /root/.cache/pip/wheels/eb/62/4b/6044430bf5a732d53bd0d1aa165c928954d9f87cd92529f00f
Successfully built standardiser


In [3]:
# Loading the data from the /data folder to a Pandas dataframe and understand which headers it has
data = pd.read_csv("/content/reference_library.csv")

In [4]:
data.head()

,smiles
0,CCCCNC(=S)N/N=C/C1=C(C)C=CS1
1,CN1C(SCC2=NC(C3=CC=CS3)=NO2)=NN=C1C1CCCCC1
2,O=C(O)CC(NC(=O)C1=CN=CC=N1)C1=CC=CC=C1Cl
3,O=S(=O)(C1=CC=CC=C1)N1CCN(C2=NOC3=CC=CC(Cl)=C2...
4,CCC[C@@H](C)N(C1=CC(Cl)=CC=C1CO)S(=O)(=O)C1=CC...


In [5]:
## Mount google drive
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Cloning the Git repository
!git clone https://github.com/KiitanTheAnalyst/Ersilia---Olaitan-Suru

Cloning into 'Ersilia---Olaitan-Suru'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 89 (delta 34), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (89/89), 247.38 KiB | 1.35 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [16]:
import sys
sys.path.append('/content/Ersilia---Olaitan-Suru/src')

from smiles_processing import standardise_smiles

smiles_list = data['smiles'].tolist()
data['standardised_smiles'] = standardise_smiles(smiles_list)

[01:54:45] Can't kekulize mol.  Unkekulized atoms: 3 7


From the above, we can see that the RDKit cound not kekulized 37 molecules

In [17]:
# Checking for Nan column
data.isna().sum()

smiles                 0
standardized_smiles    0
standardised_smiles    0
dtype: int64

In [18]:
# Dropping Nan column
data.dropna(subset=['standardised_smiles'], inplace=True)

In [19]:
# Inspect the DataFrame
data.head()

,smiles,standardized_smiles,standardised_smiles
0,CCCCNC(=S)N/N=C/C1=C(C)C=CS1,CCCCNC(=S)N/N=C/c1sccc1C,CCCCNC(=S)N/N=C/c1sccc1C
1,CN1C(SCC2=NC(C3=CC=CS3)=NO2)=NN=C1C1CCCCC1,Cn1c(SCc2nc(-c3cccs3)no2)nnc1C1CCCCC1,Cn1c(SCc2nc(-c3cccs3)no2)nnc1C1CCCCC1
2,O=C(O)CC(NC(=O)C1=CN=CC=N1)C1=CC=CC=C1Cl,O=C(O)CC(NC(=O)c1cnccn1)c1ccccc1Cl,O=C(O)CC(NC(=O)c1cnccn1)c1ccccc1Cl
3,O=S(=O)(C1=CC=CC=C1)N1CCN(C2=NOC3=CC=CC(Cl)=C2...,O=S(=O)(c1ccccc1)N1CCN(c2noc3cccc(Cl)c23)CC1,O=S(=O)(c1ccccc1)N1CCN(c2noc3cccc(Cl)c23)CC1
4,CCC[C@@H](C)N(C1=CC(Cl)=CC=C1CO)S(=O)(=O)C1=CC...,CCC[C@@H](C)N(c1cc(Cl)ccc1CO)S(=O)(=O)c1ccc(C)cc1,CCC[C@@H](C)N(c1cc(Cl)ccc1CO)S(=O)(=O)c1ccc(C)cc1


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 997 entries, 0 to 999
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   smiles               997 non-null    object
 1   standardized_smiles  997 non-null    object
 2   standardised_smiles  997 non-null    object
dtypes: object(3)
memory usage: 31.2+ KB


In [22]:
# Getting the Inchikey representation of the molecules using the RDKIT package
from smiles_processing import smiles_to_inchikey

data['inchi_key'] = data['standardised_smiles'].apply(smiles_to_inchikey)
data = data.dropna(subset=['inchi_key'])

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Charges were rearranged; Omitted undefined stereo

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Charges were rearranged

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Charges were rearranged

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Omitted undefined stereo

[01:55:09] WARNING: Charges were rearranged

[01:55:09] WARNING: Charges were rearranged

[01:55:09] WARNING: Charges were rearranged

[01:55:09] WAR